Universidad del valle de Guatemala 
Andres Quinto 
Andree Toledo 
Laboratorio 6 DS / Analisis de redes sociales

In [3]:
# #API Y TOKENS
API_KEY = "twEQRTK5ZKiol9uVpHdjzbOiY"
API_KEY_SECRET = "VB2zKLQGPsBRlQgMazxH3zZc1zSIanCadaJIlLMOhwsJsUMlPK"
TOKEN = "AAAAAAAAAAAAAAAAAAAAACjHhAEAAAAAXTh0XtmjnvybscEUdMbHmEVf%2Bh0%3DlEvp6WSu8ngO5aSqxI98QmLcRs6jviM3ZyloVqNJFKyMH9D7T0"


Hola Mundo


In [ ]:
import tweepy
import pandas as pd